In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Read Churn Dataset and get summary of basic statistics
df=pd.read_csv('./Resources/Churn_Modelling_2.csv')
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
#Remove columns not needed for analysis
df=df.drop(['RowNumber', 'CustomerId', 'Surname', 'Tenure','HasCrCard','EstimatedSalary'], axis=1)
df

,CreditScore,Geography,Gender,Age,Balance,NumOfProducts,IsActiveMember,Exited
0,619,France,Female,42,0.00,1,1,1
1,608,Spain,Female,41,83807.86,1,1,0
2,502,France,Female,42,159660.80,3,0,1
3,699,France,Female,39,0.00,2,0,0
4,850,Spain,Female,43,125510.82,1,1,0
...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,0.00,2,0,0
9996,516,France,Male,35,57369.61,1,1,0
9997,709,France,Female,36,0.00,1,1,1
9998,772,Germany,Male,42,75075.31,2,0,1


### Smoteenn Remove Age Outliers

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [6]:
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [7]:
#Create Features
X = pd.get_dummies(df, columns=['Geography', 'Gender'])
X = X.drop(["Exited"], axis=1)

# Create our target
y = df["Exited"]

In [8]:
# Check the balance of our target values
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
Counter(y_train)

Counter({1: 1540, 0: 5960})

In [10]:
# Standardize the data with StandardScaler().
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Combined over and under sampling for imbalanced classification
from imblearn.combine import SMOTEENN

# define sampling
comb_sample = SMOTEENN(random_state=42)
X_resample, y_resample = comb_sample.fit_resample(X_train, y_train)

# Check if SMOTE-ENN were properly applied
Counter(y_resample)

Counter({0: 3801, 1: 4364})

### Fit Different Module Define

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

### AdaBoost Model with SMOTEENN

In [13]:
# Initiate the model
smoteenn_adb = AdaBoostClassifier()

# Fit the model
smoteenn_adb_model = smoteenn_adb.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred = smoteenn_adb_model.predict(X_test)

In [14]:
# Calculated the balanced accuracy score
smoteenn_adb_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_adb_accuracy

0.7747729512371282

In [15]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1536,467
Actual Exited,108,389


In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn abd classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn abd classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.77      0.78      0.84      0.77      0.60      2003
          1       0.45      0.78      0.77      0.58      0.77      0.60       497

avg / total       0.84      0.77      0.78      0.79      0.77      0.60      2500



### KNN with SMOTEENN

In [17]:
# Initiate the model
smoteenn_knn = KNeighborsClassifier()

# Fit the model
smoteenn_knn_model = smoteenn_knn.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_knn_model.predict(X_test)

In [18]:
# Calculated the balanced accuracy score
smoteenn_knn_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_knn_accuracy

0.7517245258872255

In [19]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1488,515
Actual Exited,119,378


In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn knn classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn knn classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.74      0.76      0.82      0.75      0.56      2003
          1       0.42      0.76      0.74      0.54      0.75      0.57       497

avg / total       0.83      0.75      0.76      0.77      0.75      0.56      2500



### SVM with SMOTEENN

In [21]:
# Initiate the model
smoteenn_svc = SVC(kernel='rbf',probability=True)

# Fit the model
smoteenn_svc_model = smoteenn_svc.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_svc_model.predict(X_test)

In [22]:
# Calculated the balanced accuracy score
smoteenn_svc_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_svc_accuracy

0.7622992071249264

In [23]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1482,521
Actual Exited,107,390


In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn svc classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn svc classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.74      0.78      0.83      0.76      0.58      2003
          1       0.43      0.78      0.74      0.55      0.76      0.58       497

avg / total       0.83      0.75      0.78      0.77      0.76      0.58      2500



### Decision Tree with SMOTEENN

In [25]:
# Initiate the model
smoteenn_tree = DecisionTreeClassifier()

# Fit the model
smoteenn_tree_model = smoteenn_tree.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_tree_model.predict(X_test)

In [26]:
# Calculated the balanced accuracy score
smoteenn_tree_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_tree_accuracy

0.7587903858497967

In [27]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1609,394
Actual Exited,142,355


In [28]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn tree classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn tree classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.80      0.71      0.86      0.76      0.58      2003
          1       0.47      0.71      0.80      0.57      0.76      0.57       497

avg / total       0.83      0.79      0.73      0.80      0.76      0.58      2500



### Random Forest with SMOTEENN

In [29]:
# Initiate the model
smoteenn_rfc = RandomForestClassifier()

# Fit the model
smoteenn_rfc_model = smoteenn_rfc.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred = smoteenn_rfc_model.predict(X_test)

In [30]:
# Calculated the balanced accuracy score
smoteenn_rfc_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_rfc_accuracy

0.7655604118972447

In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1620,383
Actual Exited,138,359


In [32]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn rfc classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn rfc classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.81      0.72      0.86      0.76      0.59      2003
          1       0.48      0.72      0.81      0.58      0.76      0.58       497

avg / total       0.83      0.79      0.74      0.81      0.76      0.59      2500



### XGBoost Classifier with SMOTEENN

In [33]:
# Initiate the model
smoteenn_xgb = XGBClassifier()
# Fit the model
smoteenn_xgb_model =smoteenn_xgb.fit(X_resample, y_resample.ravel())
# Make Predictions
y_pred = smoteenn_xgb_model.predict(X_test)

In [34]:
# Calculated the balanced accuracy score
smoteenn_xgb_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_xgb_accuracy

0.7747513538545301

In [35]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1681,322
Actual Exited,144,353


In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn xgb classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn xgb classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.84      0.71      0.88      0.77      0.60      2003
          1       0.52      0.71      0.84      0.60      0.77      0.59       497

avg / total       0.84      0.81      0.74      0.82      0.77      0.60      2500



### Logistic Regression with SMOTEENN

In [37]:
# Initiate the model
smoteenn_lm = LogisticRegression()

# Fit the model
smoteenn_lm_model = smoteenn_lm.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_lm_model.predict(X_test)

In [38]:
# Calculated the balanced accuracy score
smoteenn_lm_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_lm_accuracy

0.7115348104603658

In [39]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1327,676
Actual Exited,119,378


In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn lm classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn lm classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.66      0.76      0.77      0.71      0.50      2003
          1       0.36      0.76      0.66      0.49      0.71      0.51       497

avg / total       0.81      0.68      0.74      0.71      0.71      0.50      2500

